In [1]:
import pandas as pd
import numpy as np
import random
import tqdm

from collections import Counter

from matplotlib import pyplot as plt
%matplotlib inline

# Advanced ML: Домашнее задание 4

В этом небольшом домашнем задании мы попробуем улучшить метод Шерлока Холмса. Как известно, в рассказе The Adventure of the Dancing Men великий сыщик расшифровал загадочные письмена.

Пользовался он для этого так называемым частотным методом: смотрел, какие буквы чаще встречаются в зашифрованных текстах, и пытался подставить буквы в соответствии с частотной таблицей: E — самая частая и так далее.
В этом задании мы будем разрабатывать более современный и продвинутый вариант такого частотного метода. В качестве корпусов текстов для подсчётов частот можете взять что угодно, но для удобства вот вам “Война и мир” по-русски и по-английски

In [2]:
with open('WarAndPeace.txt', 'rt') as f:
    war_and_peace_ru = ' '.join(line for line in f).lower()
    
with open('WarAndPeaceEng.txt', 'rt') as f:
    war_and_peace_en = ' '.join(line for line in f).lower()
    
with open('AnnaKarenina.txt', 'rt') as f:
    anna_karenina = ' '.join(line for line in f).lower()

## 1. Реализуйте базовый частотный метод по Шерлоку Холмсу:
- подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);
- возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе совсем вряд ли сработает), зашифруйте их посредством случайной перестановки символов;
- расшифруйте их таким частотным методом.

In [3]:
alphabet_en = 'abcdefghijklmnopqrstuvwxyz '
alphabet_ru = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя '

In [4]:
war_and_peace_ru = ''.join([a for a in war_and_peace_ru if a in alphabet_ru])
war_and_peace_en = ''.join([a for a in war_and_peace_en if a in alphabet_en])
anna_karenina = ''.join([a for a in anna_karenina if a in alphabet_ru])

Удалим последовательности пробелов в текстах

In [5]:
def replace_spaces(text):
    while '  ' in text:
        text = text.replace('  ', ' ')
    return text

In [6]:
war_and_peace_ru = replace_spaces(war_and_peace_ru)
war_and_peace_en = replace_spaces(war_and_peace_en)
anna_karenina = replace_spaces(anna_karenina)

In [7]:
def calculate_frequencies(text):
    c = Counter(text)
    return c

In [8]:
frequencies_wp_ru = calculate_frequencies(war_and_peace_ru)
frequencies_wp_en = calculate_frequencies(war_and_peace_en)
frequencies_ak = calculate_frequencies(anna_karenina)

In [9]:
message = 'шефредактор газеты ведомости дмитрий симаков и четыре заместителя главного редактора александр губский борис сафронов филипп стеркин и кирилл харатьян сообщили что покинут издание в знак протеста против назначения главным редактором андрея шмарова сообщается на сайте ведомостей губский работал в ведомостях с момента основания газеты все остальные порядка лет ранее ведомости сообщали что на фоне конфликта со шмаровым редакцию покинули около десяти сотрудников более чем из в том числе руководители трех отделов'
message_en = 'My name is Kvothe. I have stolen princesses back from sleeping barrow kings. I burned down the town of Trebon. I have spent the night with Felurian and left with both my sanity and my life. I was expelled from the University at a younger age than most people are allowed in. I tread paths by moonlight that others fear to speak of during day. I have talked to Gods, loved women, and written songs that make the minstrels weep. You may have heard of me. So begins a tale unequaled in fantasy literature—the story of a hero told in his own voice. It is a tale of sorrow, a tale of survival, a tale of one man’s search for meaning in his universe, and how that search, and the indomitable will that drove it, gave birth to a legend.'
# message = 'В «Анне Карениной» не было великих исторических деятелей или мировых событий. Не было здесь также лирических, философских или публицистических отступлений. Но споры, вызванные романом, сразу же вышли за пределы чисто литературных интересов, «как будто дело шло о вопросе, каждому лично близком».[8] Толки о романе Толстого сливались с злободневными политическими известиями. «О выходе каждой части Карениной, – отмечал Н.Н. Страхов, – в газетах извещают так же поспешно и толкуют так же усердно, как о новой битве или новом изречении Бисмарка».[9]«Роман очень живой, горячий и законченный, которым я очень доволен», – говорил Толстой в самом начале работы (62, 16). В последующие годы он иногда охладевал к своему роману. Но замысел «живой, горячий и законченный» вновь и вновь завладевал его воображением. А когда наконец труд многих лет был окончен, Толстой признался, что «общество, современное „Анне Карениной“, ему гораздо ближе, чем общество людей „Войны и мира“, вследствие чего ему легче было проникнуться чувствами и мыслями современников „Анны Карениной“, чем „Войны и мира“. А это имеет громадное значение при художественном изображении жизни».[10] В этом и состоит «вседневность» содержания толстовского романа «из современной жизни». Замысел Толстого, который он вначале называл «частным», постепенно углублялся. «Я очень часто, – как бы оправдываясь, признавался Толстой, – сажусь писать одно и вдруг перехожу на более широкие дороги: сочинение разрастается».[11] И успех романа оказался громадным; его читали во всех кругах образованного общества. И вскоре выяснилось, что «Анна Каренина» была встречена с осуждением в «высших сферах». М.Н. Катков решительно отказался печатать в «Русском вестнике» эпилог романа и «опустил перед Толстым шлагбаум». Уже тогда начиналось то отчуждение от дворянского высшего круга, которое позднее, после «Воскресения», привело к осуждению Толстого и отлучению его от церкви М.Н. Катков, лидер реакционной журналистики 70-х годов, тонко почувствовал острую критическую мысль Толстого и стремился всеми силами нейтрализовать впечатление, произведенное на современников «Анной Карениной». Но дело уже было сделано: Толстой высказался и облегчил свою совесть. Н.С. Лесков с тревогой отмечал, что за «Анну Каренину» Толстого дружно ругают «настоящие светские люди», а за ними «тянут ту же ноту действительные статские советники».[12] Изображение «золоченой молодежи» в лице Вронского и «сильных мира сего» в лице Каренина не могло не вызвать негодования. Сочувствие народной жизни, воплощенное в Левине и в картинах крестьянского быта, также не пробуждало восторга у «настоящих светских людей». «А небось чуют они все, – писал Толстому Фет, – что этот роман есть строгий неподкупный суд всему нашему строю жизни».[13] '

message = replace_spaces(''.join([a for a in message.lower() if a in alphabet_ru]))
message_en = replace_spaces(''.join([a for a in message_en.lower() if a in alphabet_en]))

In [10]:
def encrypt(message, alphabet):
    key = list(alphabet)
    random.shuffle(key)
    translation_table = str.maketrans(alphabet, ''.join(key))
    return message.translate(translation_table), {k: v for k, v in zip(list(alphabet), key)}

def decrypt_based_on_frequencies(message, frequencies, alphabet):
    symbols = ''.join([k for k, v in sorted(frequencies.items(), key=lambda x: x[1], reverse=True)])
    message = replace_spaces(''.join([a for a in message if a in alphabet]))
    message_frequencies = calculate_frequencies(message)
    message_symbols = ''.join([k for k, v in sorted(message_frequencies.items(), key=lambda x: x[1], reverse=True)])
    translation_table = str.maketrans(message_symbols, symbols[:len(message_symbols)])
    return message.translate(translation_table), {k: v for k, v in zip(message_symbols, symbols[:len(message_symbols)])}

def accuracy(message, decrypted_message):
    correct = 0
    for x, y in zip(message, decrypted_message):
        if x == y:
            correct += 1
    return correct / len(message)

In [11]:
print(f'Исходное сообщение:\n{message}')
encrypted, key = encrypt(message, alphabet_ru)
print(f'\nЗашифрованное сообщение:\n{encrypted}')
print('\nРасшифровка на базе частот из Войны и Мира:')
decrypted, found_key = decrypt_based_on_frequencies(encrypted, frequencies_wp_ru, alphabet_ru)
print(decrypted)
print(f'\nСовпало {accuracy(message, decrypted):0.2f} символов')

Исходное сообщение:
шефредактор газеты ведомости дмитрий симаков и четыре заместителя главного редактора александр губский борис сафронов филипп стеркин и кирилл харатьян сообщили что покинут издание в знак протеста против назначения главным редактором андрея шмарова сообщается на сайте ведомостей губский работал в ведомостях с момента основания газеты все остальные порядка лет ранее ведомости сообщали что на фоне конфликта со шмаровым редакцию покинули около десяти сотрудников более чем из в том числе руководители трех отделов

Зашифрованное сообщение:
ичпйчюоацфйхноэчцыхшчюфвфгцухювуцйусхгувоафшхухячцыйчхэовчгцуцчерхнеошёфнфхйчюоацфйохоечагоёюйхнзжгаусхжфйугхгопйфёфшхпуеу  хгцчйауёхухауйуеехдойоцлрёхгффжщуеухяцфх фауёзцхуэюоёучхшхэёоах йфцчгцох йфцушхёоэёоячёурхнеошёывхйчюоацфйфвхоёюйчрхивойфшохгффжщочцгрхёохгосцчхшчюфвфгцчсхнзжгаусхйожфцоехшхшчюфвфгцрдхгхвфвчёцохфгёфшоёурхноэчцыхшгчхфгцоелёычх фйрюаохечцхйоёччхшчюфвфгцухгффжщоеухяцфхёохпфёчхафёпеуацохгфхивойфшывхйчюоаьубх фауёзеухфа

In [12]:
print(f'Исходное сообщение:\n{message_en}')
encrypted, key = encrypt(message_en, alphabet_en)
print(f'\nЗашифрованное сообщение:\n{encrypted}')
print('\nРасшифровка на базе частот из Войны и Мира:')
decrypted, found_key = decrypt_based_on_frequencies(encrypted, frequencies_wp_en, alphabet_en)
print(decrypted)
print(f'\nСовпало {accuracy(message_en, decrypted):0.2f} символов')

Исходное сообщение:
my name is kvothe i have stolen princesses back from sleeping barrow kings i burned down the town of trebon i have spent the night with felurian and left with both my sanity and my life i was expelled from the university at a younger age than most people are allowed in i tread paths by moonlight that others fear to speak of during day i have talked to gods loved women and written songs that make the minstrels weep you may have heard of me so begins a tale unequaled in fantasy literaturethe story of a hero told in his own voice it is a tale of sorrow a tale of survival a tale of one mans search for meaning in his universe and how that search and the indomitable will that drove it gave birth to a legend

Зашифрованное сообщение:
geqizgaqjyqcshtxaqjqxzsaqythnaiqlojiwayyayqkzwcqfohgqynaaljimqkzooh qcjimyqjqkuoiadqdh iqtxaqth iqhfqtoakhiqjqxzsaqylaitqtxaqijmxtq jtxqfanuojziqzidqnaftq jtxqkhtxqgeqyzijteqzidqgeqnjfaqjq zyqablannadqfohgqtxaquijsaoyjteqztqzqehuimaoqzmaqtxziq

### Получилось как-то совсем не очень

## 2. Вряд ли в результате получилась такая уж хорошая расшифровка, разве что если вы брали в качестве тестовых данных целые рассказы. 

Но и Шерлок Холмс был не так уж прост: после буквы E, которая действительно выделяется частотой, дальше он анализировал уже конкретные слова и пытался угадать, какими они могли бы быть. Я не знаю, как запрограммировать такой интуитивный анализ, так что давайте просто сделаем следующий логический шаг:
- подсчитайте частоты биграмм (т.е. пар последовательных букв) по корпусам;
- проведите тестирование аналогично п.1, но при помощи биграмм.

In [13]:
def calculate_bigram_frequencies(text):
    c = Counter()
    for i in range(0, len(text) - 1):
        c[text[i:i+2]] += 1
    total = sum(c.values())
    for k in c:
        c[k] /= total
    return c

def decrypt_bigram(message, frequencies):
    bigrams = [k for k, v in sorted(frequencies.items(), key=lambda x: x[1], reverse=True)]
    message_bigrams = calculate_bigram_frequencies(message)
    message_sorted = [k for k, v in sorted(message_bigrams.items(), key=lambda x: x[1], reverse=True)]
    # Построим словарь для декодирования биграмм
    decoder = {k: v for k, v in zip(message_sorted, bigrams[:len(message_sorted)])}
    # Расшифруем сообщение
    result = ' '
    prev_bigram = None
    for i in range(0, len(message) - 1):
        current_bigram = message[i:i+2]
        decoded_current_bigram = decoder[current_bigram]
        # 2 буквы, которые мы хотим добавить в конец (последнюю будем перезаписывать)
        term = decoded_current_bigram
        # Проверяем должны ли мы поменять последнюю букву в существующей последовательности, или оставить ту, которая уже есть
        if prev_bigram and frequencies[prev_bigram] > frequencies[current_bigram]:
            # Если предыдущая биграмма имела частоту выше, то мы не будем менять последний символ в результате
            term = ''.join([result[-1], term[1]])
        result = ''.join([result[:-1], term])
        prev_bigram = current_bigram
    return result

In [14]:
bigram_freq_wp_ru = calculate_bigram_frequencies(war_and_peace_ru)
bigram_freq_wp_en = calculate_bigram_frequencies(war_and_peace_en)
bigram_freq_ak = calculate_bigram_frequencies(anna_karenina)

In [15]:
print(f'Исходное сообщение:\n{message}')
encrypted, key = encrypt(message, alphabet_ru)
decrypted = decrypt_bigram(encrypted, bigram_freq_wp_ru)
print(f'\nРасшифрованное сообщение:\n{decrypted}')
print(f'\nСовпало {accuracy(message, decrypted):0.2f} символов')

Исходное сообщение:
шефредактор газеты ведомости дмитрий симаков и четыре заместителя главного редактора александр губский борис сафронов филипп стеркин и кирилл харатьян сообщили что покинут издание в знак протеста против назначения главным редактором андрея шмарова сообщается на сайте ведомостей губский работал в ведомостях с момента основания газеты все остальные порядка лет ранее ведомости сообщали что на фоне конфликта со шмаровым редакцию покинули около десяти сотрудников более чем из в том числе руководители трех отделов

Расшифрованное сообщение:
свы о оо ракдевкре се  ов л дн лвамосиктай ву какр то спеегл наедстемж чсм оо оо раалллойнвойактыдмалоскрамаоихы т чвчтомомуе нэчалти юаадомегескашудти огтломока м нодлитдь кноора я иерспх  т голрх  т я ериерррур стемжесягоо оо ра олеойто свсскев о огт икоз  ероисдна се  ов нгостыдмалогоак тлегя се  ов ожима оворуылв я чввор сдевкре сзав  олыйлыа нраеутцийоогоатиа се  ов л о огт лиока м ерчсаьа й ьусаонло м всскевсягоо олсвжанодлитвов 

In [16]:
print(f'Исходное сообщение:\n{message_en}')
encrypted, key = encrypt(message_en, alphabet_en)
decrypted = decrypt_bigram(encrypted, bigram_freq_wp_en)
print(f'\nРасшифрованное сообщение:\n{decrypted}')
print(f'\nСовпало {accuracy(message_en, decrypted):0.2f} символов')

Исходное сообщение:
my name is kvothe i have stolen princesses back from sleeping barrow kings i burned down the town of trebon i have spent the night with felurian and left with both my sanity and my life i was expelled from the university at a younger age than most people are allowed in i tread paths by moonlight that others fear to speak of during day i have talked to gods loved women and written songs that make the minstrels weep you may have heard of me so begins a tale unequaled in fantasy literaturethe story of a hero told in his own voice it is a tale of sorrow a tale of survival a tale of one mans search for meaning in his universe and how that search and the indomitable will that drove it gave birth to a legend

Расшифрованное сообщение:
vesldvetlealsidnetisrf e hftitt nhanyicae howu itsa  himayanchowscfnelancotihirgees usea dne atea o nllrtantisrf e hntairdnessidseh lioitef hkesh  b it uh liohtadorve nastieh  b vewcocetiseila dnenoesitsa idnemrsta wbuiehegheiear c cheie dr s

### По метрике результат стал хуже. Похоже, что для биграмм нужен более длинный текст сообщения - все-таки у нас становится n^2 символов вместо n в словаре.

## 3. Но и это ещё не всё: биграммы скорее всего тоже далеко не всегда работают. Основная часть задания — в том, как можно их улучшить:
- предложите метод обучения перестановки символов в этом задании, основанный на MCMC-сэмплировании, но по-прежнему работающий на основе статистики биграмм;
- реализуйте и протестируйте его, убедитесь, что результаты улучшились.

Далее единицей перестановки будем считать биграмму, то есть в строке 'ABCD' - 2 биграммы - 'AB'и 'CD'. 

Предлагаю следующий метод:

- Попробуем в качестве состояний марковской цепи использовать буквы, тогда можно по корпусу построить матрицу переходов $T$. По сути это и будут биграммы.

- Далее мы можем для произвольной строки текста $s$ вычислить вероятность того, что она была порождена этой марковской цепью - $P(s) = \prod_i p(s_{i+1}|s_i) = \prod_i T_{s_i,s_{i+1}}$

- Нам нужно найти такую перестановку букв, которая после применения к зашифрованному сообщению получит результат, имеющий высокую $P$

- Попробуем применить MCMC для поиска такой перестановки, начиная от случайной:
    + На каждом шаге меняем случайно 2 буквы в перестановке местами
    + Считаем отношение $P2/P1$ для старой и новой перестановок и используя метод Метрополиса-Гастинга принимаем решение оставить ли новую перестановку ($P2 > P1$ или выпала правильная монетка) или вернуть старую. На самом деле будем брать случайную величину от 0 до 1 и сравнивать с $P2/P1$, если случайная величина больше - оставляем старую. (если $P2 > P1$ - случайная величина всегда меньше).
    + Будем делать несколько итераций (несколько сотен итераций) и сохранять результат.
    + Запускаем алгоритм много раз и выбираем лучшую финальную перестановку для всех итераций.
    
Также применил баесовское сглаживание для матрицы $T$, чтобы избежать нулевых вероятностей порождения текста из модели.

In [17]:
class CipherDecoder:
    def __init__(self, corpus, alphabet):
        self.alphabet = alphabet
        c = Counter()
        for i in range(len(corpus) - 1):
            c[(corpus[i], corpus[i + 1])] += 1
        for ch_1 in alphabet:
            for ch_2 in alphabet:
                c[(ch_1, ch_2)] += 1
        total = len(corpus) - 1 + len(alphabet) ** 2
        for k in c.keys():
            c[k] /= total
        self.T = c
    
    def decode(self, message, max_iter=2000, max_attempts=10):
        best_guess = message
        for j in range(max_attempts):
            # случайная инициализация
            permutation = self._init_permutation()
            for i in range(max_iter):
                # выбираем 2 символа перестановки, чтобы их поменять
                key_1 = random.choice(list(permutation.keys()))
                key_2 = random.choice(list(permutation.keys()))
                # расшифровка текущей перестановкой
                text_1 = self._decode_with_permutation(message, permutation)
                permutation[key_1], permutation[key_2] = permutation[key_2], permutation[key_1]
                # расшифровка измененной перестановкой
                text_2 = self._decode_with_permutation(message, permutation)
                # Вычисляем отношение для условия в алгоритме Метрополиса Гастинга (тут мы используем обратное условие, чтобы не изменять перестановку)
                ratio = self._calculate_probability_ratio(text_2, text_1)
                if random.uniform(0, 1) > ratio:
                    # Если обратное условие выполняется - отменяем изменение перестановки
                    permutation[key_1], permutation[key_2] = permutation[key_2], permutation[key_1]
            attempt_result = self._decode_with_permutation(message, permutation)
            # Получили ли мы в текущей цепочке лучший результат?
            if self._calculate_probability_ratio(attempt_result, best_guess) > 1:
                best_guess = attempt_result
        return best_guess
        
    def _decode_with_permutation(self, text, p):
        return ''.join([p[a] for a in text])
        
    def _init_permutation(self):
        # Случайная перестановка
        shuffled_alphabet = list(self.alphabet)
        random.shuffle(shuffled_alphabet)
        shuffled_alphabet = ''.join(shuffled_alphabet)
        key = {k: v for k, v in zip(self.alphabet, shuffled_alphabet)}
        return key
    
    def _calculate_probability_ratio(self, a, b):
        assert len(a) == len(b), 'Texts should be of equal length'
        ratio = 1
        for i in range(len(a) - 1):
            ratio *= self.T[(a[i], a[i+1])] / self.T[(b[i], b[i+1])]
        return ratio

In [18]:
decoder = CipherDecoder(war_and_peace_en, alphabet_en)
encrypted, key = encrypt(message_en, alphabet_en)

In [21]:
decrypted = decoder.decode(encrypted)
print(decrypted)
print(f'\nСовпало {accuracy(message_en, decrypted):0.2f} символов')

my name is kvothe i have stolen princesses back from sleeping barrow kings i burned down the town of trebon i have spent the night with felurian and left with both my sanity and my life i was expelled from the university at a younger age than most people are allowed in i tread paths by moonlight that others fear to speak of during day i have talked to gods loved women and written songs that make the minstrels weep you may have heard of me so begins a tale unequaled in fantasy literaturethe story of a hero told in his own voice it is a tale of sorrow a tale of survival a tale of one mans search for meaning in his universe and how that search and the indomitable will that drove it gave birth to a legend

Совпало 1.00 символов


Ура, расшифровка получилась, пришлос запустить несколько раз и глазами определить качество перевода, так как сходиться мржет к локальным максимумам, которые далеки от того, что мы ищем

## 4. Расшифруйте сообщение:

In [22]:
secret_message = '←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏'
secret_ru_transcoder = {k: v for k, v in zip(set(secret_message), alphabet_ru)}
secret_message_ru = ''.join([secret_ru_transcoder[k] for k in secret_message])
secret_message_ru

'ржзахвёхваоагрхлчмущзцлёихазахкчтгахлчмущзцлёихгрнжгхпхйгчъчхжччебрлафхнчгчмёихзрънчхкмчтагщгцхжнчмррхвжръчхвёхвжрхжорзщзахкмщвазцлчхахкчзптагрхущнжаущзцлёихещззхдщхкчжзролррхтргврмгчрхдщощлархнпмжщхсчгфхнчлртлчхфхлатръчхлрхчербщш'

In [23]:
print(len(set(secret_message)))

28


Различных букв 27 + пробел, то есть скорее всего язык в сообщении русский, а не английский

In [24]:
decoder_ru = CipherDecoder(war_and_peace_ru, alphabet_ru)

In [29]:
%%time
decoder_ru.decode(secret_message_ru, max_attempts=800)

CPU times: user 4min, sys: 24 ms, total: 4min
Wall time: 4min


'если вы вичите норжальный или подти норжальный текст у этого сообщения который легко продитать скорее всего вы все счелали правильно и полудите жаксижальный балл за послечнее детвертое зачание курса мотя конедно я нидего не обещах'

Теперь легко догадаться, что зашифрованное сообщение:

"если вы видите нормальный или подти нормальный текст у этого сообщения который легко прочитать скорее всего вы все сделали правильно и получите максимальный балл за последнее четвертое задание курса хотя конечно я ничего не обещаю"

## 6. Бонус: какие вы можете придумать применения для этой модели? 
Пляшущие человечки ведь не так часто встречаются в жизни (хотя встречаются! и это самое потрясающее во всей этой истории, но об этом я расскажу потом).

По идее эта модель позволяет заполнять пропуски в данных, если известна марковская цепь, которая их хорошо описывает, например:
    Восстановление пропусков в аудиозаписях (речь)
    Восстановление пропусков на изображениях